## SI Calculation: an example using generated sample trades

Here we will generate sample trades from the RTS 2 Annex III taxonomy.  Each sample trade is then enriched with the information needed run an SI calculation.

Once the trade data is assembled the the data normally provided by the regulator is synthesised.

Lastly, the SI calculations are run

The SI calculation includes a number of tests.  See the official word:
https://ec.europa.eu/transparency/regdoc/rep/3/2016/EN/3-2016-2398-EN-F1-1.PDF

# Step 1 - Prepare the trade data

The first step is to use the RTS 2 Annex III taxonomy to generate some sample trades.


In [6]:
import rts2_annex3
import random
import json

random.seed()

root = rts2_annex3.class_root

asset_class = root.asset_class_by_name("Credit Derivatives")

# Ask the Asset class to generate some sample trade
sample_trades = asset_class.make_test_samples(number=500)

# Print the one of the generated trades
print(vars(random.choice(sample_trades)))


{'notional_currency': 'notional_currency.value', 'from_date': datetime.date(2018, 4, 26), 'underlying_index': 'underlying_index.value', 'to_date': datetime.date(2018, 10, 23), 'sub_asset_class_name': 'Index credit default swap (CDS)', 'asset_class_name': 'Credit Derivatives'}


## LEIs

In a real firm with real trades we would need to know the LEI (Legal Entity Identifier) of the legal entity which did each trade because SI status is reported distinctly for each legal entity (LEI).

Quite often firms will do trades within a single legal entity, perhaps to move risk from one trading desk to another.  These are called intra-entity trades and must be filtered out before the SI calculation.  For this example we'll say that all the trades we generated are inter-entity trades, trades between distinct legal entities, so we count them all.

In this example we'll use just one LEI, and not even a valid one, but it will suffice for the example.

In [7]:
# Add our LEI to each trade
our_lei = 'Our_LEI_here'
for sample_trade in sample_trades:
    sample_trade.our_lei = our_lei

# Print the one of the modified sample trades
print(vars(random.choice(sample_trades)))

{'notional_currency': 'notional_currency.value', 'from_date': datetime.date(2018, 4, 26), 'ref_entity_type': 'ref_entity_type.value', 'our_lei': 'Our_LEI_here', 'to_date': datetime.date(2018, 10, 23), 'sub_asset_class_name': 'Single name credit default swap (CDS)', 'underlying_ref_entity': 'underlying_ref_entity.value', 'asset_class_name': 'Credit Derivatives'}


## Trade Date
The SI calculation includes checks for frequency, the number of trades done in a single week.  To work that out we need a trade date for each trade.  Here we'll just use a few dates and add these to our sample trades.

In [8]:
# We give each sample trade a trade date in a 30 day range of dates
# and an ISO week number (c.f. https://en.wikipedia.org/wiki/ISO_week_date)

import datetime

sample_dates = []
today = datetime.date.today()
for day_number in range(-30, 0):
    a_date =  today + datetime.timedelta(day_number)
    if a_date.weekday() < 6:
        sample_dates.append(a_date)

for sample_trade in sample_trades:
    selected_date = random.choice(sample_dates)
    sample_trade.trade_date = selected_date
    sample_trade.trade_date_week = selected_date.isocalendar()[1]
    
# Print the one of the modified sample trades
print(vars(random.choice(sample_trades)))

{'notional_currency': 'notional_currency.value', 'from_date': datetime.date(2018, 4, 26), 'ref_entity_type': 'ref_entity_type.value', 'our_lei': 'Our_LEI_here', 'trade_date_week': 17, 'to_date': datetime.date(2018, 10, 23), 'sub_asset_class_name': 'Single name credit default swap (CDS)', 'trade_date': datetime.date(2018, 4, 24), 'underlying_ref_entity': 'underlying_ref_entity.value', 'asset_class_name': 'Credit Derivatives'}


## MIC
The Market Identifier Code (MIC) is the ISO 10383 ID for a trading venue, for example a stock exchange.  The regulator is expected to provide a list of MIC values which identify venues which are recognised for the purposes of the SI calculation.  Trades which are done on vs. off recognised venues are counted differently.

In [9]:
# We define our MICs.  A MIC value is always 4 charcters in length.  The values used
# here are made-up nonsense, but good enough for an illustration

eea_mics = ['EEA1', 'EEA2', 'EEA3']
non_eea_mics = ['OFF1', 'OFF2', 'OFF3', 'OFF4']
all_mics = eea_mics + non_eea_mics

# Add a MIC to each sample trade
for sample_trade in sample_trades:
    sample_trade.mic = random.choice(all_mics)

# Print the one of the modified sample trades
print(vars(random.choice(sample_trades)))

{'mic': 'EEA3', 'our_lei': 'Our_LEI_here', 'trade_date_week': 13, 'sub_asset_class_name': 'Other credit derivatives', 'trade_date': datetime.date(2018, 3, 29), 'asset_class_name': 'Credit Derivatives'}


## Own Account

We need to know if a trade was done on the firms own account.  Such trades are counted differently. 

In [10]:
# Own Account is simply a boolean.  Either this is a trade which the regulator views
# as being on own account, or not.  I use a random boolean with a probability.

own_account_probability = 0.25

for sample_trade in sample_trades:
    sample_trade.own_account = random.random() < own_account_probability
    
# Print the one of the modified sample trades
print(vars(random.choice(sample_trades)))

{'notional_currency': 'notional_currency.value', 'own_account': False, 'from_date': datetime.date(2018, 4, 26), 'underlying_index': 'underlying_index.value', 'our_lei': 'Our_LEI_here', 'trade_date_week': 16, 'to_date': datetime.date(2021, 1, 20), 'sub_asset_class_name': 'Index credit default swap (CDS)', 'mic': 'OFF2', 'trade_date': datetime.date(2018, 4, 21), 'asset_class_name': 'Credit Derivatives'}


## Client Order

We need to know if a trade was done in response to a client order.  Such trades are counted differently. 

In [11]:
# Client Order is also simply a boolean.  Either this is a trade which was done
# in response to a client order, or not.  I use a random boolean.

client_order_probability = 0.5

for sample_trade in sample_trades:
    sample_trade.client_order = random.random() < client_order_probability
    
# Print the one of the modified sample trades
print(vars(random.choice(sample_trades)))

{'mic': 'EEA1', 'ref_entity_type': 'ref_entity_type.value', 'underlying_ref_entity': 'underlying_ref_entity.value', 'asset_class_name': 'Credit Derivatives', 'notional_currency': 'notional_currency.value', 'client_order': False, 'from_date': datetime.date(2018, 4, 26), 'own_account': False, 'our_lei': 'Our_LEI_here', 'trade_date_week': 14, 'sub_asset_class_name': 'Single name credit default swap (CDS)', 'trade_date': datetime.date(2018, 4, 2), 'to_date': datetime.date(2018, 10, 23)}


## EUR Notional
Another measure used by the SI calculation is the EUR notional value of each trade.  Here we assign a notional value to each trade.

In [12]:
# Add a random-ish Euro Notional amount of n million EUR to each trade
notional_amounts = [x * 1000000 for x in [1, 1, 1, 2, 2, 5, 10, 25]]

for sample_trade in sample_trades:
    sample_trade.eur_notional = random.choice(notional_amounts)

# Print one of the modified sample trades
print(vars(random.choice(sample_trades)))

{'mic': 'OFF1', 'client_order': False, 'own_account': True, 'our_lei': 'Our_LEI_here', 'trade_date_week': 15, 'sub_asset_class_name': 'Other credit derivatives', 'eur_notional': 25000000, 'trade_date': datetime.date(2018, 4, 12), 'asset_class_name': 'Credit Derivatives'}


## RTS 2 Annex III Classification
The last step before we start the SI calculation is to add the RTS 2 Annex III classification to each trade.

In [13]:
# Now classify each trade and add the JSON classification back to the trade
for sample_trade in sample_trades:
    classification = root.classification_for(subject=sample_trade)
    json_classification = json.dumps(classification.classification_dict())
    sample_trade.rts2_classification = json_classification

print(random.choice(sample_trades).rts2_classification)

{"RTS2 version": "EU 2017/583 of 14 July 2016", "Asset class": "Credit Derivatives", "Sub-asset class": "CDS index options", "Segmentation criterion 1 description": "CDS index sub-class as specified for the sub-asset class of index credit default swap (CDS )", "Segmentation criterion 1": "cds_index_sub_class.value", "Segmentation criterion 2 description": "time maturity bucket of the option defined as follows:", "Segmentation criterion 2": "Maturity bucket 4: 2 years to 3 years"}


## Put the trade data in to Pandas tables

The SI calculation requires a number of selections of the trade population.  See the comments below for details of each selection.

In [14]:
# Put the essential information for each trade into a Pandas table.

import pandas as pd

def si_details_from_sample(sample_trade):
    return dict(
        lei=sample_trade.our_lei,
        trade_date=sample_trade.trade_date,
        trade_date_week=sample_trade.trade_date_week,
        mic=sample_trade.mic,
        own_account=sample_trade.own_account,
        client_order=sample_trade.client_order,
        eur_notional=sample_trade.eur_notional,
        rts2_classification=sample_trade.rts2_classification,
        )

# The set of all trades (by LEI if there is more than one)
all_trades = pd.DataFrame.from_records([si_details_from_sample(s) for s in sample_trades])

# The subset of all trades which were not done on an EEA venue (i.e. OTC trades)
otc_trades = all_trades[~all_trades.mic.isin(eea_mics)]

# The subset of OTC trades which were done on the banks own account
own_account_otc_trades = otc_trades[otc_trades.own_account]

# The subset of own account OTC trades which were done in response to client orders
client_own_account_otc_trades = own_account_otc_trades[own_account_otc_trades.client_order]


# Step 2 - The Regulator Supplied Data

The regulator is expected to provide information about each sub class:
* Is the sub class liquid?
* How many trades of that sub class were done in the whole EU?
* What is the total EUR notional value traded in that sub class in the whole EU?

We don't have any regulator supplied data here so we synthesise some.

In [15]:
# For every RTS 2 sub class we need to decide if it is liquid or not
# We simply generate a random true/false for each sub class and use
# a dictionary to hold the result so we can look it up later.

distinct_sub_classes = all_trades.rts2_classification.unique()
liquidity_dictionary = dict()
for sub_class in distinct_sub_classes:
    is_liquid = random.random() < 0.5
    liquidity_dictionary[sub_class] = is_liquid
liquidity_dictionary.values()


dict_values([True, False, True, False, False, True, True, True, False, True, False, True])

In [16]:
# The values which will be compared with the EU trade count and sum(eur_notional)
# are the counts and totals of the own account OTC trades which were done in 
# response to client orders.  We synthesise the test EU numbers from these.

# First we get the counts and sum of notional, grouping by sub class (RTS 2 string)
notional_by_sub_class = client_own_account_otc_trades[['rts2_classification', 'eur_notional']]\
    .groupby(by='rts2_classification')
sums_series = notional_by_sub_class.agg(['count', 'sum'])
sums_df = pd.DataFrame(sums_series)

sums_df.columns = sums_df.columns.get_level_values(0)
sums_df = sums_df.reset_index()
sums_df.columns = pd.Index(
    ['rts2_classification', 'trade_count', 'trade_notional_sum'], 
    dtype='object')

# We add a column for the EU trade count for each sub class.  For this exercise, the
# threshold for being an SI is if our LEO count for the subclass is >= 2.5% of
# the EU count.  The EU figure is randomly set to be a bit more or a bit less than
# will trigger SI status.

sums_df['eu_count'] = sums_df['trade_count']\
    .apply(lambda x: x * 40 + random.choice([x * -1, x]) )

# Add a column for the EU notional for each sub class.  The threshold for
# notional is 1% of the EU figure.  Again the EU number is randomly tweaked
sums_df['eu_eur_notional'] = sums_df['trade_notional_sum']\
    .apply(lambda x: x * 100 + random.choice([x * -1, x]) )


# Add a column for the average number of trades per week
min_week_number = all_trades['trade_date_week'].min()
max_week_number = all_trades['trade_date_week'].max()
number_of_weeks = max_week_number - min_week_number + 1
sums_df['avg_weekly_trades'] = sums_df['trade_count']\
    .apply(lambda x: x /  number_of_weeks)

# Add a column which indicates if the subclass is liquid
sums_df['is_liquid'] = sums_df['rts2_classification']\
    .apply(lambda x: liquidity_dictionary[x] )
    
# Add a column which is the total notional of all trades by the LEI
# TODO: This is picking up the same sum as the client-own-otc ... why?
all_notional_by_sub_class = all_trades[['rts2_classification', 'eur_notional']]\
    .groupby(by='rts2_classification')
all_sums_series = notional_by_sub_class.sum()
all_sums_df = pd.DataFrame(all_sums_series)
all_sums_df = all_sums_df.reset_index()
all_sums_df.columns = pd.Index(
    ['rts2_classification', 'all_trade_notional_sum'], 
    dtype='object')

# Join the all sums df with the client-own_account-otc table
sums_df = pd.merge(sums_df, all_sums_df, how='left', on='rts2_classification')

sums_df.head(2)

,rts2_classification,trade_count,trade_notional_sum,eu_count,eu_eur_notional,avg_weekly_trades,is_liquid,all_trade_notional_sum
0,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",4,22000000,156,2178000000,0.8,False,22000000
1,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",5,10000000,205,990000000,1.0,True,10000000
2,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",2,2000000,82,202000000,0.4,False,2000000
3,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",3,17000000,123,1717000000,0.6,True,17000000
4,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",3,40000000,123,3960000000,0.6,True,40000000
5,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",1,2000000,39,198000000,0.2,False,2000000
6,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",5,20000000,205,2020000000,1.0,True,20000000
7,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",8,39000000,312,3939000000,1.6,True,39000000
8,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",2,4000000,78,396000000,0.4,False,4000000
9,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",1,1000000,41,99000000,0.2,True,1000000


# Step 3 - Do the SI calculation

The "calculation" is really a set of filters which might catch an RTS 2 subclass for an LEI

1. If the RTS 2 Annex III sub class is liquid
   - and the count of client own-account otc trades >= 2.5% of eu_rts2_trade_count
   - and average weekly number of client own-account otc trades >= 1
2. If the RTS 2 Annex III sub class is not liquid 
   - and average weekly number of client own-account otc trade >= 1
3. If the sum of EUR notional for client own-account otc trades is
   - \>= 25% of all trades notional for the LEI
   - **or** >= 1% of EU trade notional

In [17]:
# Filter 1:  This is looking at the trades we have in client_own_account_otc_trades.
# We need to join the liquidity, EU trade count and weekly trade avereages data 
# with trades data frame, then we can for which the following three things are true:
#    The RTS 2 sub class is liquid
#    The count of our trades for that sub class >= 2.5% of the EU count
#    More than one trade per week was done on average

x1 = sums_df.copy()
x1[(x1.is_liquid) 
   & (x1.trade_count >= (x1.eu_count * 0.025))
   & (x1.avg_weekly_trades >= 1)]


,rts2_classification,trade_count,trade_notional_sum,eu_count,eu_eur_notional,avg_weekly_trades,is_liquid,all_trade_notional_sum
7,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",8,39000000,312,3939000000,1.6,True,39000000


In [22]:
# Filter 2 just looks at the average weekly number of trades for non liquid sub classes

x2 = sums_df.copy()
x2[(~x2.is_liquid) 
   & (x2.avg_weekly_trades >= 1)]


,rts2_classification,trade_count,trade_notional_sum,eu_count,eu_eur_notional,avg_weekly_trades,is_liquid,all_trade_notional_sum


In [24]:
# Filter 3

x3 = sums_df.copy()
x3[  (x3.trade_notional_sum >= (x3.all_trade_notional_sum * 0.25))
   | (x3.trade_notional_sum >= (x3.eu_eur_notional * 0.01))]

# Remove this once the all_trades total notional bug is fixed
x3.head(2)

,rts2_classification,trade_count,trade_notional_sum,eu_count,eu_eur_notional,avg_weekly_trades,is_liquid,all_trade_notional_sum
0,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",4,22000000,156,2178000000,0.8,False,22000000
1,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",5,10000000,205,990000000,1.0,True,10000000


# An Object Oriented Calculator

Here we build a tiny Python application to do the SI calculation on the generated trades.

The result of the calculation is a JSON report of the RTS 2 Annex III sub classes for which our LEI is a Systematic Internaliser.

We define 3 classes:
* SIReport - This represents the report spanning all RTS 2 classes for one LEI
* RTS2SubClass - This represents the infirmation for just one RTS 2 sub class
* Aggregations - Represents the various counts and sums for one RTS 2 sub class

If you want to see the report, scroll past the code to the next text box.

In [136]:
import collections

class SIReport(object):
    
    @classmethod
    def for_trades(cls, trades):
        new_report = cls()
        new_report.add_trades(trades)
        return new_report
    
    def __init__(self):
        self.trades = []
        self.sub_classes = dict()
        self._number_of_weeks = None
        
    def add_trades(self, trades):
        self.trades.extend(trades)
        for trade in trades:
            rts2_string = trade.rts2_classification
            if not rts2_string in self.sub_classes:
                self.sub_classes[rts2_string] = RTS2SubClass(self, rts2_string)
            sub_class = self.sub_classes[rts2_string]
            sub_class.add_trade(trade)
        self._number_of_weeks = None

    @property
    def number_of_weeks(self):
        if self._number_of_weeks is None:
            min_trade = min(self.trades, key=lambda t: t.trade_date_week)
            max_trade = max(self.trades, key=lambda t: t.trade_date_week)
            self._number_of_weeks = max_trade.trade_date_week - \
                                    min_trade.trade_date_week + 1
        return self._number_of_weeks

    def si_sub_classes(self):
        return [sub_class for sub_class in self.sub_classes.values() 
                if sub_class.si_status()]
        
    def report(self):
        si_sub_classes = self.si_sub_classes()
        report_items = [
            'This LEI is an SI for {si_count} of {all_count} '
            'sub classes traded over {weeks} weeks.'.format(
                si_count=len(si_sub_classes),
                all_count=len(self.sub_classes),
                weeks=self.number_of_weeks)]
        for sub_class in si_sub_classes:
            report_item = ['Status: {status}.'.format(
                    status=sub_class.si_status())]
            agg_dict = vars(sub_class.aggregations)
            del agg_dict['sub_class']
            del agg_dict['trades']
            report_item.append(vars(sub_class.aggregations))
            report_item.append(json.loads(
                    sub_class.rts2_string, object_pairs_hook=collections.OrderedDict))
            report_items.append(report_item)
        return json.dumps(report_items, indent=4)


class RTS2SubClass(object):
    def __init__(self, report, rts2_string):
        self.report = report
        self.rts2_string = rts2_string
        self.is_liquid = random.random() < 0.5
        self.trades = []
        self._aggregations = None

    @property
    def aggregations(self):
        # I keep all the computed results in one object so I can drop the cache
        # if a trade is added
        if self._aggregations is None:
            self._aggregations = Aggregations(sub_class=self)
        return self._aggregations
        
    def add_trade(self, trade):
        self.trades.append(trade)
        self._aggregations = None

    def si_status(self):
        """
        This is the SI Calculation.  It is applied distinctly to each sub class.
        It's quite simple once everything is aggregated.
        Note that these are the rules for derivatives trades only.
        """
        agg = self.aggregations
        if self.is_liquid \
            and agg.trade_count >= (0.025 * agg.eu_trade_count) \
            and agg.avg_weekly_trades >= 1:
                return "SI - Liquid instrument test"
        if not self.is_liquid \
            and agg.avg_weekly_trades >= 1:
                return "SI - Non-liquid instrument test"
        if agg.notional_sum >= (0.25 * agg.sub_class_trades_notional_sum) \
            or agg.notional_sum >= (0.01 * agg.eu_notional_sum):
                return "SI - Notional size test"
        return None

    
class Aggregations(object):
    """
    Each instance of Aggregations represents the subset of the trades for
    a sub class which are OTC client orders on the own account of the LEI.
    The SI calculation tests are with respect to this subset of the trades.
    """

    def __init__(self, sub_class):
        self.sub_class = sub_class
        
        # Build the aggregations for this sub class
        self.trades = [
                trade for trade in self.sub_class.trades
                if (not trade.mic in eea_mics)  # OTC
                    and trade.own_account       # Traded on own account
                    and trade.client_order]     # in response to a client order
        self.trade_count = len(self.trades)
        self.notional_sum = sum([abs(trade.eur_notional) for trade in self.trades])
        self.avg_weekly_trades = self.trade_count / self.sub_class.report.number_of_weeks
        
        # Now I synthesise the EU figures which should really come from the regulator
        self.eu_trade_count = self.trade_count * 40 + random.choice([
                                self.trade_count * -1, self.trade_count])
        self.eu_notional_sum  = self.notional_sum * 100 + random.choice([
                                    self.notional_sum * -1, self.notional_sum])

        # I keep this sub class sum here so it gets flushed if trades added
        self.sub_class_trades_notional_sum = sum(
                [abs(trade.eur_notional) for trade in self.sub_class.trades])


## Run the report
Having defined the classes we should be able to just run the report.

Note that the result is different every time the report is run because the EU totals are re-synthesised each time and are random.  With real data the report would be stable, and indeed this test could be changed to always produce the same results; the current implementaion is intended to show some variety.

In [150]:
# First create an instance of our report
report = SIReport.for_trades(sample_trades)

# Then get the JSON report and print it
print(report.report())


[
    "This LEI is an SI for 6 of 12 sub classes traded over 5 weeks.",
    [
        "Status: SI - Notional size test.",
        {
            "sub_class_trades_notional_sum": 537000000,
            "trade_count": 4,
            "eu_notional_sum": 2178000000,
            "avg_weekly_trades": 0.8,
            "eu_trade_count": 156,
            "notional_sum": 22000000
        },
        {
            "RTS2 version": "EU 2017/583 of 14 July 2016",
            "Asset class": "Credit Derivatives",
            "Sub-asset class": "Bespoke basket credit default swap (CDS)"
        }
    ],
    [
        "Status: SI - Notional size test.",
        {
            "sub_class_trades_notional_sum": 401000000,
            "trade_count": 3,
            "eu_notional_sum": 3069000000,
            "avg_weekly_trades": 0.6,
            "eu_trade_count": 117,
            "notional_sum": 31000000
        },
        {
            "RTS2 version": "EU 2017/583 of 14 July 2016",
            "Asset class": "Cr

# A Declarative Calculator

Here we use Pandas to run the calculation in a more declarative, relational kind of way.

First we pick up the EU data from the OO model so the results here will be the same as the results in the OO report above.

In [151]:
def eu_data_for_sub_class(sub_class):
    return dict(
        rts2_classification=sub_class.rts2_string,
        is_liquid=sub_class.is_liquid,
        eu_trade_count=sub_class.aggregations.eu_trade_count,
        eu_trade_notional=sub_class.aggregations.eu_notional_sum,
        )

# The set of all trades (by LEI if there is more than one)
eu_sub_class_data = pd.DataFrame.from_records([eu_data_for_sub_class(s) \
                                     for s in list(report.sub_classes.values())])
#len(report.sub_classes)
eu_sub_class_data.head(2)

,eu_trade_count,eu_trade_notional,is_liquid,rts2_classification
0,117,1717000000,False,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""..."
1,156,2178000000,False,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""..."


In [154]:
# Get the sum of all trades by RTS 2 sub class.
# This should exactly match the figure in the OO report.

all_trades_notional_sum_series = all_trades[['rts2_classification', 'eur_notional']]\
    .groupby(by='rts2_classification').sum()
sums_df = pd.DataFrame(all_trades_notional_sum_series).reset_index()
# Rename the column to all_trades_eur_notional
sums_df

,rts2_classification,eur_notional
0,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",537000000
1,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",285000000
2,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",56000000
3,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",185000000
4,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",375000000
5,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",96000000
6,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",565000000
7,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",367000000
8,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",73000000
9,"{""RTS2 version"": ""EU 2017/583 of 14 July 2016""...",78000000
